In [0]:
%run ./01-streaming-word-count  

In [0]:
class WordCountTestSuite:
    def __init__(self, spark):
        self.base_dir = "/Volumes/workspace/default/spark_streaming"
        self.spark = spark
    
    def cleanTests(self):
        print("Performing cleanup tasks")
        self.spark.sql("drop table if exists word_counts")
        print("Done!")
    
    def ingestData(self, itr):
        print("Starting ingestion ...")
        dbutils.fs.cp(f"{self.base_dir}/datasets/text/text_data_{itr}.txt", f"{self.base_dir}/data/text/")
        print("Done!")
    
    def assertResult(self, expected_count):
        actual_count = self.spark.sql("select sum(count) from word_counts where substr(word, 1, 1) == 's'").collect()[0][0]
        assert actual_count == expected_count, "Expected %d, found %d" % (expected_count, actual_count)
    
    def runTests(self):
        self.cleanTests()
        wc = StreamingWordCount(spark)

        print("Testing first iteration of the batch word count")
        self.ingestData(1)
        wc.wordCount()
        self.assertResult(25)
        print("First iteration complete")

        print("Testing second iteration of the batch word count")
        self.ingestData(2)
        wc.wordCount()
        self.assertResult(32)
        print("Second iteration complete")


In [0]:
wc_test_suite = WordCountTestSuite(spark)
wc_test_suite.runTests()